In [ ]:
# TODO
# https://github.com/mwcraig/ipyevents click and key cluster choice
# https://scikit-network.readthedocs.io/en/latest/reference/hierarchy.html
#     sknetwork.hierarchy.cut_balanced is it worth anything?
# krakow balance param - plot how it affects clustering quality

In [ ]:
%load_ext lab_black
import networkx as nx
import community as community_louvain
import numpy as np
from time import time
from IPython.core.display import display, HTML
from IPython.display import Image
from scipy.cluster.hierarchy import cut_tree, to_tree, leaves_list
from ipywidgets import Button, HBox, VBox, Output, Layout

from paris import paris
from paris_utils import plot_dendrogram
from import_and_scrape_helpers import scrape
from helpers import (
    save_graph,
    load_graph,
    id_to_url,
    id_to_thumbnail,
    get_liked_from_time_range,
)

# import matplotlib.cm as cm
# import matplotlib.pyplot as plt

In [ ]:
# temporary autocompletion bug fix
%config Completer.use_jedi = True

In [ ]:
G = load_graph()

In [ ]:
def sort_nodes_by_in_degree(G, nodes):
    recs = sorted(G.subgraph(nodes).in_degree(), key=lambda pair: pair[1], reverse=True)
    ids, scores = zip(*recs)
    return ids, scores


# def recommend(SubG, pickiness=0):
#     # this line recommends very normie videos
#     # it's equivalent to pickiness==0
#     # recs = sorted(SubG.in_degree(), key=lambda pair: pair[1], reverse=True)

#     # to be honest, I don't fully understand this part
#     # but it works better than the one on top:
#     # first limit recs only to the best ones
#     # this way, we'll omit most general normie recommendations later
#     best_recs = [node for node, in_degree in SubG.in_degree() if in_degree >= pickiness]
#     #     best_recs = [node for node, degree in SubG.degree() if degree >= pickiness]

#     ids, scores = sort_nodes_by_in_degree(SubG, best_recs)
#     return ids, scores


def display_video_links(G, ids, prefixes=None, show_first=15):
    if prefixes is None:
        prefixes = [""] * len(ids)

    for id_, prefix in list(zip(ids, prefixes))[:show_first]:
        if "title" not in G.nodes[id_] or G.nodes[id_]["title"] is None:
            scrape(id_, G)
            if "title" not in G.nodes[id_]:
                # scrapig failed - probably video is down
                continue

        url = id_to_url.format(id_)
        title = G.nodes[id_]["title"]

        display(
            Image(
                url=id_to_thumbnail.format(id_),
                width=300,
            )
        )
        display(HTML(f"""<a href="{url}">{prefix} {title}</a>"""))
        print()
        print()


#     save_graph(G)

In [ ]:
seconds_in_year = 60 * 60 * 24 * 365
start_time = time() - seconds_in_year * 3
sources = get_liked_from_time_range(G, start_time)
print("sources: ", len(sources))

RecentDirected = G.edge_subgraph(G.out_edges(sources))

# RecentDirectedNeighborhood = G.edge_subgraph(G.out_edges(RecentDirected.nodes))
# Recent = RecentDirectedNeighborhood.to_undirected()
Recent = RecentDirected.to_undirected()

print("number of videos: ", Recent.number_of_nodes())

In [ ]:
# choose only the biggest connected component
components = sorted(nx.connected_components(Recent), key=len, reverse=True)
# for el in components[:5]:
#     print(len(el))
main_component = components[0]
Main = Recent.subgraph(main_component)

In [ ]:
D = paris(Main)

In [ ]:
# plot_dendrogram(D)

In [ ]:
tree = to_tree(D)

main_ids_list = np.array(Main.nodes)

In [ ]:
left_out = Output()
right_out = Output()

layout = Layout(width="300px", height="40px")  # set width and height
left_button = Button(layout=layout)
right_button = Button(layout=layout)

out = HBox(
    [
        VBox([left_button, left_out], layout=Layout(width="300px")),
        VBox([right_button, right_out], layout=Layout(width="300px")),
    ],
)

In [ ]:
def update_displayed_videos():
    global left, right
    left = tree.get_left()
    right = tree.get_right()

    left_leaves = left.pre_order()
    right_leaves = right.pre_order()

    left_out.clear_output(wait=True)
    right_out.clear_output(wait=True)
    with left_out:
        cluster = main_ids_list[left_leaves]
        ids, scores = sort_nodes_by_in_degree(RecentDirected, cluster)
        display_video_links(G, ids)  # , scores)
    with right_out:
        cluster = main_ids_list[right_leaves]
        ids, scores = sort_nodes_by_in_degree(RecentDirected, cluster)
        display_video_links(G, ids)  # , scores)

In [ ]:
def left_clicked(b):
    global tree
    tree = left
    update_displayed_videos()


def right_clicked(b):
    global tree
    tree = right
    update_displayed_videos()


left_button.on_click(left_clicked)
right_button.on_click(right_clicked)

update_displayed_videos()

In [ ]:
out